In [1]:
import os 

import os.path as osp

import shutil
os.chdir(osp.dirname(os.getcwd()))


In [2]:
from data.model_util_scannet import ScannetDatasetConfig
import numpy as np 
import sys
from src.arkitscenes_dataset import ARKitSceneDataset
from loguru import logger

from my_script.pc_utils import *
from my_script.utils import *

from src.joint_det_dataset import Joint3DDataset




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


WARNING - 2022-10-19 14:29:15,079 - collision - No FCL -- collision checking will not work
WARNING - 2022-10-19 14:29:15,212 - svg_io - SVG path loading unavailable!


In [3]:
def get_attr(data):
    return list(data.keys())

def print_attr_shape(data):
    for k,v in data.items():
        if   isinstance(v,str):
            
            logger.info(f"{k} : {v}")
        elif   isinstance(v,bool):
            logger.info(f"{k} : {v}")
        elif   isinstance(v,int):
            logger.info(f"{k} : {v}")
        else :
            logger.info(f"{k} : {v.shape}")
            
train_dataset = Joint3DDataset(
            dataset_dict={'sr3d':1,'scannet':10},
            test_dataset='sr3d', #? only test set need ? 
            split='train',
            use_color=True, use_height=False,
            overfit=True,
            data_path='datasets/',
            detect_intermediate=True,#? 
            use_multiview=False, #? 
            butd=False,
            butd_gt=False,
            butd_cls=True,
            augment_det=False
        )

length = len(train_dataset.annos)
for idx in range(length):
    data = train_dataset.__getitem__(idx)
    if len(data['utterances'].split('.'))>4:
        break

Loading train files, take a breath!


In [5]:
# prompt_demo_idx = 1280 
# train_dataset.__getitem__(prompt_demo_idx)['utterances']
# get_attr(data)
# print_attr_shape(data)
# print(f" box num : {(data['box_label_mask']==1).sum()}")
# print(f" label number :  {len(data['utterances'].split('.'))}")


In [8]:
DATASET_CONFIG = ScannetDatasetConfig()

point_num = 50000

TRAIN_DATASET_WK = ARKitSceneDataset('train', num_points=point_num,
                                        augment=True,
                                        start_proportion=0.0,
                                        end_proportion=1.0,data_root='datasets/ARKitScenes')



# TEST_DATASET_ARKIT = ARKitSceneDataset('valid', num_points=point_num,
#                                             augment=False,
#                                             start_proportion=0.0,
#                                             end_proportion=1.0,)



Find 4364 in ARKitScene dataset!


In [12]:

#? vertex_normals 
#? heading_class_label 
#? heading_residual_label 
#? num_gt_boxes 
demo = TRAIN_DATASET_WK.__getitem__(0)

logger.warning(get_attr(demo))
print_attr_shape(demo)

#* for test 
# demo_test = TEST_DATASET_ARKIT.__getitem__(0)
# logger.warning(get_attr(demo_test))
# logger.info("================================================================================")
# print_attr_shape(demo_test)


2022-10-19 10:18:26.269 | WARNING  | __main__:<module>:7 - ['scan_ids', 'point_clouds', 'ema_point_clouds', 'flip_x_axis', 'flip_y_axis', 'rot_mat', 'scale', 'all_bboxes', 'all_bbox_label_mask', 'all_class_ids', 'is_view_dep', 'is_hard', 'is_unique', 'num_gt_boxes', 'supervised_mask']
2022-10-19 10:18:26.275 | INFO     | __main__:print_attr_shape:8 - scan_ids : 40753679
2022-10-19 10:18:26.276 | INFO     | __main__:print_attr_shape:14 - point_clouds : (50000, 6)
2022-10-19 10:18:26.276 | INFO     | __main__:print_attr_shape:14 - ema_point_clouds : (50000, 6)
2022-10-19 10:18:26.277 | INFO     | __main__:print_attr_shape:14 - flip_x_axis : ()
2022-10-19 10:18:26.278 | INFO     | __main__:print_attr_shape:14 - flip_y_axis : ()
2022-10-19 10:18:26.278 | INFO     | __main__:print_attr_shape:14 - rot_mat : (3, 3)
2022-10-19 10:18:26.278 | INFO     | __main__:print_attr_shape:14 - scale : ()
2022-10-19 10:18:26.279 | INFO     | __main__:print_attr_shape:14 - all_bboxes : (64, 6)
2022-10-19 1

(72727, 6)


In [15]:





def save_(data):
    FLAG = "debug"  #* for teacher or student 
    make_dirs(osp.join(debug_path,data['scan_ids']))

    write_pc_as_ply(
                data['point_clouds'],
                os.path.join(debug_path,data['scan_ids'], '%s_gt_%s.ply'%(data['scan_ids'],FLAG))
            )

    #* open3d  format bounding box 
    np.savetxt(os.path.join(debug_path, data['scan_ids'],'%s_box_%s.txt'%(data['scan_ids'],FLAG)),
            data['all_bboxes'][data['all_bbox_label_mask']],fmt='%s')



debug_path = "logs/debug"
make_dirs(debug_path)
save_(demo)



In [7]:





def is_exists(path):
    return osp.exists(path)

def get_new_path(data_root,scene_name):
    return osp.join(data_root,scene_name,f'{scene_name}_offline_prepared_data_new')



def delete_new_dir():
    for scene_name in os.listdir(data_path):
        # print(get_file(data_path,scene_name))
        new_path = get_new_path(data_path,scene_name)
        
        print(new_path)
        # print(os.listdir(new_path))
        if is_exists(new_path):
            shutil.rmtree(new_path)



def copyBox2NewDir(data_dir,scene_name):
    src_file = osp.join(data_dir,f'{scene_name}_offline_prepared_data',f"{scene_name}_bbox.npy")
    if osp.exists(src_file):
        shutil.copy(src_file,osp.join(data_dir,f'{scene_name}_offline_prepared_data_new'))
    
    
BASE_DIR="datasets/ARKitScenes"
split= "train"

if split== "train":
    data_path = os.path.join(BASE_DIR, 'dataset', "3dod/Training")
else :
    data_path = os.path.join(BASE_DIR, 'dataset', "3dod/Validation")

scan_ids = open(osp.join(BASE_DIR,'dataset',f"{split}_filtered.txt")).read().strip().split("\n")

for scene_name in scan_ids:
    copyBox2NewDir(osp.join(data_path,scene_name),scene_name)

    
    
    